Neste projeto, desenvolvido no âmbito da disciplina CCF425 - Introdução à Ciência
dos Dados, buscamos aplicar os conceitos teóricos aprendidos em sala de aula para analisar
conjuntos de dados reais e desafiadores. O trabalho está dividido em cinco etapas, cada uma
com objetivos específicos, culminando em uma apresentação final que destaca as principais
descobertas.
Nesta primeira etapa, nosso foco é o entendimento inicial dos dados e sua preparação.
Trabalharemos com um dos conjuntos de dados disponíveis: Dados de criminalidade de SP
(SPSafe) ou Dados demográficos dos municípios brasileiros (BrStats), dependendo da turma
prática. Nosso grupo ficou responsável pelo conjunto de dados do SPSafe. O objetivo é
explorar os atributos presentes, identificar possíveis ruídos, tratar informações ausentes e
formular pelo menos 10 perguntas que guiarão nossas análises futuras. Além disso,
realizaremos ajustes na formatação dos dados, enriquecimento com atributos externos e
outras atividades necessárias para garantir a qualidade da base de dados.
A entrega desta etapa inclui a criação de um projeto no GitHub, onde disponibilizaremos um
arquivo README com os integrantes do grupo, as perguntas elaboradas, o código utilizado e
um relatório parcial documentando as decisões tomadas e suas justificativas. Cada integrante
contribuirá ativamente, e suas ações serão registradas no histórico de commits e na
documentação.
Esta fase é fundamental para estabelecer as bases do projeto, garantindo que os dados
estejam prontos para as análises exploratórias e técnicas mais avançadas que serão aplicadas
nas etapas subsequentes. Com um trabalho bem estruturado desde o início, estaremos
preparados para enfrentar os desafios típicos da Ciência de Dados e extrair insights valiosos
dos conjuntos de dados selecionados.


**Papéis de cada integrante**
Todos os integrantes tiveram papel fundamental no decorrer do trabalho até aqui, seja
contribuindo com as ideias e perguntas, ou seja contribuindo com desenvolvimento dos
códigos. A seguir detalharemos o papel de cada integrante:

➢ Arthur Teodoro: Participou ativamente no desenvolvimento e decisão das das
perguntas, na elaboração do relatório e foi responsável pela implementação da adição
dos dados externos de “Datas Atípicas”.

➢ Gabriel Benez: Participou na criação e decisão de quais perguntas selecionar, na
elaboração do relatório e também foi responsável pela a implementação do código que
preenche os dados vazios do campo “Cidade”, e detecção de alguns ruídos.

➢ Lucas Fonseca: Participou ativamente no desenvolvimento e decisão das das
perguntas, na elaboração do relatório e foi responsável pela implementação e melhora da adição
dos dados externos de “Incidentes”, remoção de dados dublipcados,adição de correlação e gráficos de;
"Quantidade de BO's por dia na semana", "Relação quantidades BO's com periodo do Dia",
"Dias da semana têm maior incidência de invasões domiciliares (Pergunta 8)", associatividade entre tipo de crimes e o local de alaboração do BO, e associação de datas de jogos de futebol com tipo de crimes.

➢ Matheus Kauan: Responsável pela criação do repositório e adição dos
colaboradores. Participou ativamente no desenvolvimento e nas escolhas das
perguntas e também trabalhou na adição/criação de uma nova coluna no dataframe
inserindo os dias da semana correspondente a data

In [ ]:
import subprocess
import sys
import json
import pandas as pd
import numpy as np
from shapely.geometry import Point
import math
import geopandas as gpd
import re
from rapidfuzz import process, fuzz
import unicodedata
import matplotlib.pyplot as pl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import plotly.express as px

<H2 align="center"> Baixar o arquivo csv, caso não tenha baixado o arquivo via github:</H2>


In [ ]:

# Tenta importar gdown, se não conseguir, instala
try:
    import gdown  # Tenta importar novamente após a instalação
except ImportError:
    print("📥 gdown não encontrado. Instalando automaticamente...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])

# Baixar o arquivo do Google Drive

file_id = "1VbB_fvowkgNoWcLADi1mEfFjWQI6nBjp"
url = f"https://drive.google.com/uc?id={file_id}"
output = "dados/SpSafe_2022.csv"

gdown.download(url, output, quiet=False)

<p align="center">
  Leitura do arquivo
</p>

In [ ]:

df = pd.read_csv('dados/SpSafe_2022.csv', delimiter=';')
pd.set_option('display.max_columns', None)

Verificando os diferentes tipos de crimes registrados:

In [ ]:
df['NATUREZA_APURADA'].dropna().unique()

Dados nulos para profissão

Pensamos em analisar a correlação das vítimas e suas profissões, mas grande parte dos dados são nulos.

In [ ]:
porcentagem_nan_prof = df['PROFISSAO'].isna().mean() * 100
print(f"{porcentagem_nan_prof:.2f}% dos dados são NaN")

Criação do dicionário dos campos:

In [ ]:
dados_boletim = {
    "Nome do Campo": [
        "NUM BO", "ANO BO", "CODIGO BOLETIM", "NATUREZA APURADA", "DATA OCORRENCIA", "HORA OCORRENCIA", "PERIODO OCORRENCIA",
        "CIDADE", "COD IBGE", "LOGRADOURO", "NUMERO LOGRADOURO", "BAIRRO", "UF", "TIPO LOCAL", "LATITUDE", "LONGITUDE", "PONTO CRIME",
        "DELEGACIA ELABORACAO", "DEPARTAMENTO ELABORACAO", "SECCIONAL ELABORACAO", "TIPO PESSOA", "GENERO PESSOA", "IDADE PESSOA",
        "DATA NASCIMENTO PESSOA", "COR PELE", "PROFISSAO", "PLACA VEICULO", "UF VEICULO", "CIDADE VEICULO", "COR VEICULO",
        "MARCA VEICULO", "MODELO VEICULO", "ANO FABRICACAO", "ANO MODELO", "TIPO VEICULO", "MARCA CELULAR", "QUANT CELULAR",
        "BO INICIADO", "BO EMITIDO", "DATA HORA ELABORACAO", "DATA COMUNICACAO", "BO AUTORIA", "FLAGRANTE", "EXAME", "SOLUCAO",
        "ESPECIE", "STATUS", "FLAG VITIMA FATAL", "DESDOBRAMENTO"
    ],
    "Descrição do Campo": [
        "Número do boletim de ocorrência", "Ano do boletim de ocorrência", "Junção do número com o ano do BO separados por '/'",
        "Tipo de crime cometido", "Data em que o crime ocorreu", "Hora em que o crime ocorreu", "Período do dia em que o crime ocorreu",
        "Cidade em que o crime ocorreu", "Código da cidade (IBGE)", "Via em que o crime ocorreu",
        "Número do local do crime", "Bairro do crime", "UF do crime", "Tipo de local do crime",
        "Latitude do ponto do crime", "Longitude do ponto do crime", "Ponto geográfico no formato WKT",
        "Delegacia que elaborou o BO", "Departamento que elaborou o BO", "Seccional que elaborou o BO",
        "Indica se é vítima ou autor", "Gênero", "Idade", "Data de nascimento", "Cor da pele", "Profissão",
        "Placa do veículo", "UF do emplacamento", "Cidade do emplacamento", "Cor do veículo",
        "Marca do veículo", "Modelo do veículo", "Ano de fabricação", "Ano do modelo", "Tipo de veículo",
        "Marca do celular", "Quantidade de celulares", "Data/hora em que o BO foi iniciado", "Data/hora em que o BO foi concluído",
        "Data/hora de elaboração do BO", "Data em que o BO foi comunicado", "Responsável pelo BO",
        "Indica se foi flagrante", "Responsável pelo exame", "Tipo de solução do crime",
        "Espécie de patrimônio", "Status do crime", "Indica se houve vítima fatal", "Desdobramento do caso"
    ]
}

df_dicionario = pd.DataFrame(dados_boletim)
df_dicionario

Para entendermos se há alguma correlação entre o dia da semana e os crimes cometidos, foi criada uma nova coluna com os dias da semana que obtivemos a partir da data da ocorrência do crime. 

<H2 align="center"> Códigos criados para adicionar uma coluna com os atributos dia da semana. </H2>
 

<P aling = "center">  </P>
<p align="center">
  Executar somente quando um novo arquivo for baixado do Google Drive
</p>



In [ ]:
#Conventendo a coluna DATA_OCORRENCIA para datetime para ser reconhecido como data pelo pandas
df['DATA_OCORRENCIA'] = pd.to_datetime(df['DATA_OCORRENCIA'])

In [ ]:
#Criação de coluna com o dia da semana
df.insert(df.columns.get_loc('DATA_OCORRENCIA') + 1, 'DIA_SEMANA', df['DATA_OCORRENCIA'].dt.day_name())

In [ ]:
#Convertendo os dias da semana para português
dias_semana = {
    'Monday': 'SEGUNDA-FEIRA', 'Tuesday': 'TERÇA-FEIRA', 'Wednesday': 'QUARTA-FEIRA',
    'Thursday': 'QUINTA-FEIRA', 'Friday': 'SEXTA-FEIRA', 'Saturday': 'SÁBADO', 'Sunday': 'DOMINGO'
}
df['DIA_SEMANA'] = df['DATA_OCORRENCIA'].dt.day_name().map(dias_semana)

In [ ]:
df.head(20)

<H2 align="center"> Códigos criados para verificar se a data teve um evento atípico. </H2>
 

Devido há vários eventos e datas comemorativas que ocorrem durante o ano, um novo atributo será adicionado para entendermos se há uma correlação entre a data específica e quantidade de crimes cometidos. 

<p align="center">
  Executar somente para o arquivo csv baixado via Google Drive
</p>


In [ ]:
#Definir os eventos de São Paulo em 2022
eventos_sp_2022 = [
    #Feriados em Sp 2022:
    '2022-01-01',  # Confraternização Universal
    '2022-04-15',  # Paixão de Cristo
    '2022-04-21',  # Tiradentes
    '2022-05-01',  # Dia do Trabalho
    '2022-09-07',  # Independência do Brasil
    '2022-10-12',  # Nossa Senhora Aparecida
    '2022-11-02',  # Finados
    '2022-11-15',  # Proclamação da República
    '2022-12-25',  # Natal
    '2022-07-09',  # Revolução Constitucionalista
    '2022-02-28',  # Carnaval
    '2022-03-01',  # Carnaval
    '2022-06-16',  # Corpus Christi
    '2022-10-28',  # Dia do Servidor Público
    
    #Facultativos:
    '2022-03-02',  # Quarta-Feira de Cinzas
    '2022-12-24',  # Véspera de Natal
    '2022-12-31',   # Véspera de Ano Novo

    '2022-01-25',  # Final da Copa São Paulo de Futebol Júnior
    '2022-03-10',  # Palmeiras x São Paulo - Paulista
    '2022-03-30',  # Final Paulista - Jogo 1
    '2022-04-03',  # Final Paulista - Jogo 2
    '2022-05-02',  # São Paulo x Santos - Brasileirão
    '2022-05-22',  # Corinthians x São Paulo - Brasileirão
    '2022-06-20',  # São Paulo x Palmeiras - Brasileirão
    '2022-08-21',  # Santos x São Paulo - Brasileirão
    '2022-09-11',  # São Paulo x Corinthians - Brasileirão
    '2022-10-16',  # Palmeiras x São Paulo - Brasileirão
    
    # Jogos da Copa do Mundo 2022 (Seleção Brasileira)
    '2022-11-24',  # Brasil 2 x 0 Sérvia
    '2022-11-28',  # Brasil 1 x 0 Suíça
    '2022-12-02',  # Camarões 1 x 0 Brasil
    '2022-12-05',  # Brasil 4 x 1 Coreia do Sul (Oitavas)
    '2022-12-09'   # Brasil 1(2) x (4)1 Croácia (Quartas)
]

In [ ]:
#Conventendo a coluna DATA_OCORRENCIA para datetime para ser reconhecido como data pelo pandas
df['DATA_OCORRENCIA'] = pd.to_datetime(df['DATA_OCORRENCIA'])

# Converter para datetime para comparação
eventos_dates = pd.to_datetime(eventos_sp_2022)

In [ ]:
#Criar nova coluna 'DIA ATIPICO' que indica se na data aconteceu um evento atípico:
df['DIA ATIPICO'] = df['DATA_OCORRENCIA'].isin(eventos_dates)

#Converter para texto mais descritivo
df['DIA ATIPICO'] = df['DIA ATIPICO'].map({True: 'SIM', False: 'NÃO'})

In [ ]:
#df.head(20)

# Filtrar o DataFrame para retornar apenas as linhas onde 'DIA ATIPICO' é 'SIM'
df_atipico = df[df['DIA ATIPICO'] == 'SIM']
# Exibir o resultado
df_atipico

Ao analisarmos o dataset, detectamos alguns problemas com certas informações que apresentam dados nulos: falta de nomes das cidades e ausência de períodos.

### Tratamento de PERIODO_OCORRENCIA:

Verificando Objetos que 'PERIODO_OCORRENCIA' são "NaN":

In [ ]:
df_nam = df[df['PERIODO_OCORRENCIA'].isna()]
df_nam

Verificando Objetos que 'PERIODO_OCORRENCIA' e 'HORA_OCORRENCIA' são "NaN":

In [ ]:
mask1 = df['HORA_OCORRENCIA'].isna().astype(bool)
mask2 = df['PERIODO_OCORRENCIA'].isna().astype(bool)
df_nam = df[mask1 & mask2]  # or mask1 | mask2 for OR condition
df_nam

Filtro de Tratamento

In [ ]:

# 1. Filtra linhas onde PERIODO_OCORRENCIA é nulo (usando isna)
df_nam = df[df['PERIODO_OCORRENCIA'].isna()].copy()

# 2. Atualiza PERIODO_OCORRENCIA baseado em HORA_OCORRENCIA (formato "HH:MM:SS")
def definir_periodo(hora_str):
    if pd.isna(hora_str):
        return "EM HORA INCERTA"
    
    try:
        hora_part = hora_str.split(':')[0]
        hora = int(hora_part)
    except (ValueError, IndexError, AttributeError):
        return "EM HORA INCERTA"
    
    if 0 <= hora < 5:
        return "DE MADRUGADA"
    elif 5 <= hora < 12:
        return "PELA MANHÃ"
    elif 12 <= hora < 18:
        return "A TARDE"
    elif 18 <= hora <= 23:
        return "A NOITE"
    else:
        return "EM HORA INCERTA"

# Aplica a função para atualizar os valores
df_nam['PERIODO_OCORRENCIA'] = df_nam['HORA_OCORRENCIA'].apply(definir_periodo)

# Verifica o resultado
df_nam

Atualiza o DataFrame:

In [ ]:
df['PERIODO_OCORRENCIA'] = np.where(
    df['PERIODO_OCORRENCIA'].isna(),
    df['HORA_OCORRENCIA'].apply(definir_periodo),
    df['PERIODO_OCORRENCIA']
)

df[df['PERIODO_OCORRENCIA'].isna()]

### Incidentes:

Criação do campo:

In [ ]:

# Criar uma coluna combinada para o local de elaboração
df['LOCAL_ELABORACAO'] = df['DELEGACIA_ELABORACAO'].fillna(
                          df['DEPARTAMENTO_ELABORACAO'].fillna(
                          df['SECCIONAL_ELABORACAO']))

# Definir as colunas para agrupar os incidentes
dados_incidente = ['DATA_OCORRENCIA', 'PERIODO_OCORRENCIA', 
                   'LOGRADOURO', 'NUMERO_LOGRADOURO', 'BAIRRO', 
                   'UF', 'TIPO_LOCAL', 'LOCAL_ELABORACAO','DIA_SEMANA']

# Criar os IDs de incidentes com base no agrupamento
df['INCIDENTES_ID'] = df.groupby(dados_incidente, dropna=False).ngroup() + 1
df

In [ ]:
#Incidentes sem Id
df[df['INCIDENTES_ID'].isna()]

Quantidade de Incidentes:

In [ ]:
# Filtrar apenas registros que pertencem a incidentes (excluindo possíveis registros sem agrupamento)
dados_com_incidentes = df[df['INCIDENTES_ID'].notna()]

# Mostrar esses dados
dados_com_incidentes.nunique()

Revisão da análise inicial e tratamentos:

Atributos externos adicionados
Para responder todas estas questões será necessário um pouco mais que os dados
disponibilizados no Sp Safe, por este motivo, adicionaremos mais informações que serão
essenciais para a análise e entendimento dos dados. As novas adições foram:

1) Dias da semana:
Como buscamos traçar um padrão de comportamento baseado em períodos da semana, foi
necessário transcrever as datas dos ocorridos para seus respectivos dias da semana. Para isso
utilizamos “df['DATA_OCORRENCIA'].dt.day_name()”, que vai extrair da data da
ocorrência o seu dia da semana, possibilitando fazer a análise de um possível padrão semanal.

2) Eventos atípicos:
Entender se há uma relação entre crimes ocorridos e dias “não rotineiros” faz parte dos
inúmeros objetivos deste projeto, logo, precisamos conhecer as datas de grandes eventos que
movimentaram a população no ano de 2022. Para isso adicionamos um conjunto de datas
marcantes no ano de 2022, como, por exemplo, feriados relevantes e jogos importantes.
Usando estes dados exteriores podemos identificar crimes que ocorreram em datas e eventos
movimentados e entender como se correlacionam.

3) Incidentes:
Com a finalidade de agrupar os crimes juntos, podemos observar que no banco de dados pode
ser que um crime tenha ocorrido como uma invasão de propriedade e tenha ocorrido um
furto. Esses dois crimes são considerados distintos, um sendo de invasão de propriedade e o
outro de furto. Então, é interessante criar um novo atributo que identifique incidentes. Se
houver informações iguais em outros atributos, significa que provavelmente são do mesmo
incidente, ocorreram ao mesmo tempo e com a mesma pessoa.
Com o tratamento de dados atual, que ainda apresenta muitos registros em branco, foi
possível associar um número de incidente a 20.556 BOs (Boletins de Ocorrência). No
entanto, o dataset completo contém 720.446 registros. Essa diferença evidencia a necessidade
de complementar as informações faltantes, pois só assim será possível determinar se um
3
determinado BO ou crime faz parte de um conjunto de ocorrências relacionadas a um mesmo
incidente.


**Tratamento de ausência de dados**
Para responder as perguntas escolhidas precisaríamos da completude de alguns campos, o que
nem sempre ocorre. Veja algumas destas demandas e suas soluções abaixo:
1) Falta de nomes de cidades:
Para resolver este problema, usamos a biblioteca “geopy” para recuperar o nome da cidade
onde o crime descrito ocorreu usando sua latitude e longitude. Essa biblioteca pega os valores
de latitude e longitude, e por meio de uma API, recupera o endereço onde essas coordenadas
batem, retornando a informação faltante no campo “cidade”.
2) Ausência de períodos:
Alguns registros de BO não possuem seus períodos registrados de acordo seus respectivos
horários, apresentando valores nulos (NaN). Com base nas informações do SPSafe, o turnos
do periodo foram adicionados a partir desses horários registrados, foi feito, primeiramente,
filtrando o dataframe para exibir as linhas de dados em que esse fator ocorre e em seguida foi
aplicada a lógica para adição dos turnos. Após o tratamentos desses dados nulos, foram
acrescentadas mais de 6 mil novas linhas sem dados nulos na coluna de
PERIODO_OCORRENCIA.


**Ruídos detectados:**
1) Erro na digitação ou formatação dos dados de latitude e longitude.
Foi detectado alguns erros na digitação dos dados do campo “latitude” e “longitude” do
dataframe. Um desses erros foi o valor numérico que não corresponde ao padrão usado
para identificar as coordenadas, e deslocamento da vírgula.
2) BO’s com idade igual a 0:
Foi detectado também alguns BO's em que o autor/vítima possui 0 anos de idade

<h2 align="center">
  Códigos criados para localizar as cidades dos dados via latitude e longitude
</h2>


<div style = "text-align: center;">
<strong>Diferente da primeira entrega, usamos o geopanda para mapear as cidades via latitude e longitude usando shapefile da Fundação Seade (Sistema Estadual de Análise de Dados)</strong>
<br>
<a href="https://portalgeo.seade.gov.br/">Clique aqui para visitar o site</a>

</div>


In [ ]:
malha = gpd.read_file("dados/Shapefile/LimiteMunicipal.shp")

<div style = "text-align: center;">
<p> Percorremos cada linha, cruzando os dados do dataframe SpSafe com o que foi gerado com o shapefile da base geográfica dos municípios de São Paulo </p>
</div>


In [ ]:
contador_erro = 0
for indice, linha in df.iloc[1:].iterrows():
	if pd.isna(linha["LATITUDE"]) and pd.isna(linha["LONGITUDE"]):
		continue
	if pd.isna(linha["CIDADE"]) and pd.notna(linha["LATITUDE"]) and pd.notna(linha["LONGITUDE"]):
		ponto = Point(linha["LONGITUDE"], linha["LATITUDE"])
		cidade = malha[malha.geometry.contains(ponto)]
		if not cidade.empty:
			df.at[indice, "CIDADE"] = cidade.iloc[0]["Municipio"]
		else:
			contador_erro = contador_erro + 1
			df.iloc[indice, "LATITUDE", "LONGITUDE"]
	if(contador_erro >= 10 and contador_erro % 10 == 0):
		print(f"{contador_erro} erros.")

df.to_csv('dados/SpSafe_2022.csv', index=False, sep=';', encoding='utf-8')
print("Gravado")


<div style = "text-align: center;">
<h2> Tratamentos de erros</h2>
</div>


<div style = "text-align: center;">
<p> Erros por latitude com virgula em posição incorreta </p>
</div>


In [ ]:
def verificaMag(valor):
    if valor == 0:
        return 0
    return math.floor(math.log10(abs(valor))) - 1

cont_alteracoes = 0

for indice, linha in df.iloc[1:].iterrows():
    if(pd.isna(linha["CIDADE"]) and pd.notna(linha["LATITUDE"]) and pd.notna(linha["LONGITUDE"])):
        lat = linha["LATITUDE"] / 10 ** verificaMag(linha["LATITUDE"])
        long = linha["LONGITUDE"] / 10 ** verificaMag(linha["LONGITUDE"])
        ponto = Point(long, lat)
        cidade = malha[malha.geometry.contains(ponto)]

        if not cidade.empty:
            df.at[indice, "CIDADE"] = cidade.iloc[0]["Municipio"]
            cont_alteracoes = cont_alteracoes + 1


print(f"Foram feitas {cont_alteracoes} alteraçoes")
df.to_csv('dados/SpSafe_2022.csv', index=False, sep=';', encoding='utf-8')
    

<div style = "text-align: center;">
<p> Erros por falta de latitude e longitude </p>
<p> Tratamento correlacionando com um DF de delegacias -> cidades encontrada via secretária de segurança pública de São Paulo </p>
<a href="https://portalgeo.seade.gov.br/">Clique aqui para visitar o site</a>
</div>


In [ ]:
delegacias = df["DELEGACIA_CIRCUNSCRICAO"].unique()
print(delegacias)

<div style = "text-align: center;">
<p> Abaixo vamos fazer uma pre-normalização  a escrita do dataFrame de delegacias, tirando simbolos, acentos</p>
<p> A biblioteca unicodedate foi responsável por essa normalização </p>
</div>

In [ ]:

def remove_simbolos(texto):
    texto = str(texto)

    # Remove símbolos específicos antes da normalização
    texto = texto.replace("º", "")  # ordinal masculino
    texto = texto.replace("ª", "")  # ordinal feminino
    texto = texto.replace("°", "")  # grau

    # Normaliza (remove acentos)
    texto = unicodedata.normalize("NFKD", texto)

    # Remove qualquer caractere que não seja letra, número ou espaço
    texto = ''.join(c for c in texto if c.isalnum() or c.isspace())

    return texto

df["DELEGACIA_ELABORACAO"] = df["DELEGACIA_ELABORACAO"].apply(remove_simbolos)

print(remove_simbolos("02º D.P. JACAREIº"))

<div style = "text-align: center;">
<p> Agora usamos uma biblioteca chamada RapidFuzz, para encontrar casamentos por similaridades usando o algoritimo
    distancia Levenshtein, que classifica as similaridade de duas strings em um valor númerico de 0 a 100  </p>
<p> Também usamos a biblioteca re, que trabalha com substituição/remoção de padrões 
</div>

In [ ]:


df_delegacias = pd.read_csv("dados/DADOS CRIMINAIS.csv", delimiter=',')

def preprocessar_string(s):
    s = s.lower()
    s = re.sub(r'[^\w\s]', '', s)  # [] Toda palavra ^negação \w Letras, números e underscore \s espaço
    s = re.sub(r'\s+', ' ', s).strip()
    s = re.sub(r'^0+(\d+)', r'\1', s)
    return s

# Preprocessa todas as delegacias
df_delegacias["DP_PREP"] = df_delegacias["DP"].astype(str).apply(preprocessar_string)

def procura_similaridade(delegacia):
    s = preprocessar_string(str(delegacia))
    match = process.extractOne(
        s,
        df_delegacias["DP_PREP"],
        scorer=fuzz.ratio
    )
    if match and match[1] >= 90:
        idx = df_delegacias["DP_PREP"][df_delegacias["DP_PREP"] == match[0]].index[0]
        return df_delegacias.loc[idx, "MUNICIPIO"]
    return np.nan

# Aplica nos registros com cidade nula
mascara = df["CIDADE"].isna()
df.loc[mascara, "CIDADE"] = df.loc[mascara, "DELEGACIA_ELABORACAO"].apply(procura_similaridade)

print("Substituições concluídas.")

<div style = "text-align: center;">
<p> Ruidos de cidades com valor 2513851.0, que corresponde a cidade de Santo André </p>
</div>


In [ ]:
df.loc[df["CIDADE"] == "2513851.0", "CIDADE"] = "Santo Andre"

df.to_csv('dados/SpSafe_2022.csv', index=False, sep=';', encoding='utf-8')
print("Dados atualizados com sucesso.")

<div style = "text-align: center;">
<p> Algumas correções que faltaram  com  maiores repetições </p>
</div>


In [ ]:
delegacias_agrupadas = df[df["CIDADE"].isnull()]["DELEGACIA_ELABORACAO"].value_counts()
delegacias_agrupadas

In [ ]:

for indice, linha in df[df["CIDADE"].isnull()].iterrows():
    if linha["DELEGACIA_ELABORACAO"] == "73 DP JACANA":
        df.at[indice, "CIDADE"] = "São Paulo"
    elif linha["DELEGACIA_ELABORACAO"] == "PLANTAO DE SOROCABA  Z NORTE":
        df.at[indice, "CIDADE"] = "Sorocaba"
    elif linha["DELEGACIA_ELABORACAO"] == "CPJ RIBEIRAO PRETO":
        df.at[indice, "CIDADE"] = "Ribeirão Preto"
    elif linha["DELEGACIA_ELABORACAO"] == "DELPOLPLANTAO SUMARE":
        df.at[indice, "CIDADE"] = "Sumaré"
    elif linha["DELEGACIA_ELABORACAO"] == "101 DP JDIM IMBUIAS":
        df.at[indice, "CIDADE"] = "Jardim das Imbuias"
    elif linha["DELEGACIA_ELABORACAO"] == "89 DP JARDIM TABOAO":
        df.at[indice, "CIDADE"] = "São Paulo"
    elif linha["DELEGACIA_ELABORACAO"] == "DELSECFRANCA PLANTAO":
        df.at[indice, "CIDADE"] = "Franca"
    elif linha["DELEGACIA_ELABORACAO"] == "DELSECITANHAEM PLANTAO":
        df.at[indice, "CIDADE"] = "Itanhaém"
    elif linha["DELEGACIA_ELABORACAO"] == "CPJ LINS":
        df.at[indice, "CIDADE"] = "Lins"
    elif linha["DELEGACIA_ELABORACAO"] == "27 DP DR IGNACIO FRANCISCO":
        df.at[indice, "CIDADE"] = "São Paulo"
    elif linha["DELEGACIA_ELABORACAO"] == "DELSECRIO CLARO PLANTAO":
        df.at[indice, "CIDADE"] = "Rio Claro"

print("Alterações realizadas com sucesso.")

In [ ]:
df.to_csv('dados/SpSafe_2022.csv', index=False, sep=';', encoding='utf-8')
print("Salvo com sucesso")

<div style = "text-align: center;">
<h2> Cruzamento de dados: SpSafe com os Dados demográficos</h2>
<p> Cruzei os dados das cidades conseguidas a cima, com os dados demográficos do estado de São Paulo via SEADE </p>
<a href="https://repositorio.seade.gov.br/dataset/demografia">Clique aqui para visitar o site</a>

</div>

In [ ]:
malha_demografica = gpd.read_file("dados/Dados Demograficos/Den Demografica.shp")

<div style = "text-align: center;">
<p> Algumas correções de cidades incorretas, e igualando padrões com os Dados Demográficos   </p>
</div>

In [ ]:
df["CIDADE"] = df["CIDADE"].replace({
    "Jardim das Imbuias": "São Paulo",
    "Santa Bárbara D'Oeste" : "Santa Bárbara d'Oeste",
    "Águas de Lindoia" : "Águas de Lindóia",
    "Pompeia" : "Pompéia", 
    "Santa Rosa do Viterbo" : "Santa Rosa de Viterbo",
    "Uchôa" : "Uchoa",
    "Santo Andre": "Santo André"
})
df.to_csv('dados/SpSafe_2022.csv', index=False, sep=';', encoding='utf-8')
print("Alterações feitas com sucesso.")

In [ ]:
df_cidades = df["CIDADE"].value_counts().reset_index()
df_cidades.columns = ["CIDADE", "QUANTIDADE_CRIMES"]
df_cidades["QUANTIDADE_POPULACAO"] = 0

for indice, linha in df_cidades.iterrows():
    qtd = malha_demografica[malha_demografica["Nome"].str.contains(linha["CIDADE"], na=False)]

    if not qtd.empty:
        df_cidades.at[indice, "QUANTIDADE_POPULACAO"] = qtd.iloc[0]["Pop"]


df_cidades

In [ ]:
quantidade_crime = df_cidades["QUANTIDADE_CRIMES"].sum()
quantidade_pessoas = df_cidades["QUANTIDADE_POPULACAO"].sum()
media_estadual = round(quantidade_crime / quantidade_pessoas * 100000)


print(f"Quantidade crime {quantidade_crime}, Quantidade pessoas {quantidade_pessoas}")

print(f"A média estadual é de {media_estadual} crimes a cada 100 mil habitantes. ")

In [ ]:
df_cidades["CRIMES_POR_100K"] = round((df_cidades["QUANTIDADE_CRIMES"] / df_cidades["QUANTIDADE_POPULACAO"]) * 100000)
print("Dados calculados com sucesso.")

In [ ]:
df_cidades_filtrado = df_cidades.loc[(df_cidades.QUANTIDADE_POPULACAO > 0 ) & (df_cidades.QUANTIDADE_POPULACAO < 50000 )]

In [ ]:
df_cidades_filtrado

In [ ]:

# Tamanho do gráfico
plt.figure(figsize=(12, 6))

# Scatter plot
plt.scatter(df_cidades_filtrado["QUANTIDADE_POPULACAO"], df_cidades_filtrado["CRIMES_POR_100K"], color='dodgerblue', alpha=0.7)

# Linha horizontal com a média
plt.axhline(y=media_estadual, color='red', linestyle='--', label=f'Média estadual: {media_estadual:.2f}')

# Labels e título
plt.xlabel("População")
plt.ylabel("Crimes por 100 mil habitantes")
plt.title("Relação entre Media de crimes em cidades com população menor que 50k com a estadual")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


<div style = "text-align: center;">
<p> Tabela com os dados do gráfico acima.</p>
</div>

In [ ]:
df_cidades_filtrado.loc[df_cidades_filtrado.CRIMES_POR_100K > 1431]

<div style = "text-align: center;">
<h2> Tratamento de BO's Duplicados</h2>
<p> </p>
</div>

<div style = "text-align: left;">
<p> Definindo de quais serão considerados como BO's Duplicados, a partir de uma lista de parametros contendo as colunas (campos) que serão utilizadas para a definição de um filtro. 

Utilizar somente o campo 'NUM_BO' não é interssante,pois apenas considerando tal campo se tem uma maior probabilidade de exclusão de dados não duplicados. Já que com este pensamento não é levado em conta a possibilidade de erros de digitação ou outros possiveis erros dos quais no final resultariam na exclusão de dados não duplicados, e sim dados que precisam, somente, de um tratamento.</p>
</div>

In [ ]:
param_duplicados = ['NUM_BO','NATUREZA_APURADA' ,'CODIGO_BOLETIM', 'LOCAL_ELABORACAO','TIPO_LOCAL']

# Identifica duplicatas (mantendo o primeiro como "não duplicado")
duplicados = df.duplicated(subset=param_duplicados, keep='first')

# Exibe os duplicados
df_duplicados = df[duplicados]
df_duplicados

<div style = "text-align: center;">
<h3> Remoção dos dados duplicados do Dataframe:</h3>
</div>

In [ ]:
# Remove os duplicados do DataFrame original
df_sem_duplicados = df[~duplicados]
df = df[~duplicados]
df

<div style = "text-align: center;">
<p>Calculo de média estadual de crimes por 100 mil habitantes.</p>
</div>

In [ ]:
quantidade_crime = df_cidades["QUANTIDADE_CRIMES"].sum()
quantidade_pessoas = df_cidades["QUANTIDADE_POPULACAO"].sum()
media_estadual = round(quantidade_crime / quantidade_pessoas * 100000)


print(f"Quantidade crime {quantidade_crime}, Quantidade pessoas {quantidade_pessoas}")

print(f"A média estadual é de {media_estadual} crimes a cada 100 mil habitantes. ")

In [ ]:
df_cidades["CRIMES_POR_100K"] = round((df_cidades["QUANTIDADE_CRIMES"] / df_cidades["QUANTIDADE_POPULACAO"]) * 100000)
print("Dados calculados com sucesso.")

<div style = "text-align: center;">
<p>Cidades com menos de 50.0000 habitantes.</p>
</div>

In [ ]:
df_cidades_filtrado = df_cidades.loc[(df_cidades.QUANTIDADE_POPULACAO > 0 ) & (df_cidades.QUANTIDADE_POPULACAO < 50000 )]

<div style = "text-align: center;">
<p>Gráfico de dispersão das cidades com população menor de 50k, comparando a taxa de crime por 100k pessoas, com a média estadual</p>
</div>

In [ ]:
df_cidades_filtrado

In [ ]:

# Tamanho do gráfico
plt.figure(figsize=(12, 6))

# Scatter plot
plt.scatter(df_cidades_filtrado["QUANTIDADE_POPULACAO"], df_cidades_filtrado["CRIMES_POR_100K"], color='dodgerblue', alpha=0.7)

# Linha horizontal com a média
plt.axhline(y=media_estadual, color='red', linestyle='--', label=f'Média estadual: {media_estadual:.2f}')

# Labels e título
plt.xlabel("População")
plt.ylabel("Crimes por 100 mil habitantes")
plt.title("Relação entre Media de crimes em cidades com população menor que 50k com a estadual")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

<div style = "text-align: center;">
<p>Tabela com as cidades;</p>
</div>

In [ ]:
df_cidades_filtrado.loc[df_cidades_filtrado.CRIMES_POR_100K > 1431]

<div style = "text-align: center;">
<h3> Quantidade de BO's por dia na semana</h3>
</div>

<div style = "text-align: left;">
<p> Interesse em em entender quais dias da semana, e sua proporção, acontecem maior quantidade de crimes, a partir da quantidade de BO's registrados. Assim, podendo inferir em quais dias são mais prováveis que ocorra crimes, junto de sua proproção, no estado de São Paulo.</p> 
</div>

In [ ]:

dias = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado", "Domingo"]
crimes = [0] * 7
dias_semana = ["SEGUNDA-FEIRA", "TERÇA-FEIRA", "QUARTA-FEIRA", "QUINTA-FEIRA", "SEXTA-FEIRA", "SÁBADO", "DOMINGO"]

for i, dia in enumerate(dias_semana):
    crimes[i] = df.loc[
        (df['DIA_SEMANA'] == dia)
    ].shape[0]

media = sum(crimes) / len(crimes)


# Gráfico
plt.figure(figsize=(12, 6))
cores = plt.cm.viridis([i / 6 for i in range(7)])
barras = plt.bar(dias, crimes, color=cores)

plt.xlabel("Dias da Semana", fontsize=14)
plt.ylabel("Quantidade de BO's", fontsize=14)
plt.title("Quantidade de BO's por dia na semana", fontsize=16, weight='bold')

for barra in barras:
    altura = barra.get_height()
    plt.text(barra.get_x() + barra.get_width()/2., altura + 0.5,
             f'{int(altura)}', ha='center', va='bottom', fontsize=12)

# Linha da média
plt.axhline(media, color='red', linestyle='--', linewidth=2, label=f'Média: {media:.1f}')
plt.text(len(dias) - 0.5, media + 0.5, f'', color='red', fontsize=12)

# Grade, legenda e layout
plt.grid(True, axis='y', linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()

<div style = "text-align: center;">
<h3> Relação quantidades BO's com periodo do Dia</h3>
</div>

<div style = "text-align: left;">
<p> Com os dados sobre a quantidade de B.O.s registrados por dia da semana, tornou-se interessante também identificar os períodos com maior incidência de crimes, ou seja, os momentos em que há mais registros de B.O.s. Dessa forma, buscou-se correlacionar a quantidade de B.O.s com o período em que os crimes ocorreram, permitindo uma análise mais precisa dos momentos de maior criminalidade.</p>
</div>

In [ ]:

# Mapeamento dos períodos
periodo_mapeamento = {
    'PELA MANHA': 'Manhã',
    'A TARDE': 'Tarde',
    'A NOITE': 'Noite',
    'DE MADRUGADA': 'Madrugada',
    'EM HORA INCERTA': 'Incerto'
}

# Aplica o mapeamento para uma nova coluna
df['PERIODO_SIMPLIFICADO'] = df['PERIODO_OCORRENCIA'].map(periodo_mapeamento)

# Períodos simplificados na ordem desejada
periodos = ["Manhã", "Tarde", "Noite", "Madrugada", "Incerto"]
qnt_BO = []

# Contagem de BOs por período
for periodo in periodos:
    contagem = df[df['PERIODO_SIMPLIFICADO'] == periodo].shape[0]
    qnt_BO.append(contagem)

# Cálculo da média
media = sum(qnt_BO) / len(qnt_BO)

# Gráfico
plt.figure(figsize=(12, 6))
barras = plt.bar(periodos, qnt_BO)

# Labels e título
plt.xlabel("Períodos", fontsize=14)
plt.ylabel("Quantidade de BO's", fontsize=14)
plt.title("Quantidade de BO's por Período do Dia", fontsize=16, weight='bold')

# Adiciona os valores em cima de cada barra
for barra in barras:
    altura = barra.get_height()
    plt.text(barra.get_x() + barra.get_width()/2., altura + 0.5,
             f'{int(altura)}', ha='center', va='bottom', fontsize=12)

# Linha da média
plt.axhline(media, color='red', linestyle='--', linewidth=2, label=f'Média: {media:.1f}')
plt.text(len(periodos) - 0.5, media + 1, f'', color='red', fontsize=12)

# Grade, legenda e layout
plt.grid(True, axis='y', linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()

<div style = "text-align: center;">
<h3> Dias da semana têm maior incidência de invasões domiciliares (Pergunta 8)</h3>
</div>

<div style = "text-align: left;">
<p> Necessidade dee criar uma lista de tipos de crimes que podem ser relacionados com invasão domiciliar para que tenha se uma maior amostragem de dados que se encaxem com invasão domiciliar.

Porém, a amostragem teve uma pequena quantidade de dados, mesmo utilizando a lista de tipos de crimes. Outro ponto para observação é pelo fato de serem crimes relacionados há a possibilidade de não refletir com uma boa acurácia (ruído) a resposta da pergunta</p>
</div>

In [ ]:
crimes_invasao_domiciliar = [
    'VIOLACAO DE DOMICILIO (ART. 150)',
    'FURTO (ART. 155) - RESIDENCIA',
    'FURTO QUALIFICADO (ART. 155, §4O.) - RESIDENCIA',
    'A.I.-FURTO (ART. 155) - RESIDENCIA',
    'A.I.-FURTO QUALIFICADO (ART. 155, §4O.) - RESIDENCIA',
    'FURTO DE COISA COMUM (ART. 156) - RESIDENCIA',
    'ROUBO (ART. 157) - RESIDENCIA'
]

df[df['NATUREZA_APURADA'].isin(crimes_invasao_domiciliar)]

<div style = "text-align: center;">
<h3> Geração do Gráfico</h3>
</div>

In [ ]:

# Dados
dias = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado", "Domingo"]
invasoes = [0] * 7
dias_semana = ["SEGUNDA-FEIRA", "TERÇA-FEIRA", "QUARTA-FEIRA", "QUINTA-FEIRA", "SEXTA-FEIRA", "SÁBADO", "DOMINGO"]

for i, dia in enumerate(dias_semana):
    invasoes[i] = df.loc[
        (df['DIA_SEMANA'] == dia) & (df['NATUREZA_APURADA'].isin(crimes_invasao_domiciliar))
    ].shape[0]

# Cálculo da média
media = sum(invasoes) / len(invasoes)


# Gráfico
plt.figure(figsize=(12, 6))
cores = plt.cm.viridis([i / 6 for i in range(7)])
barras = plt.bar(dias, invasoes, color=cores)

# Labels e título
plt.xlabel("Dias da Semana", fontsize=14)
plt.ylabel("Invasões Domiciliares", fontsize=14)
plt.title("Invasões Domiciliares por Dia da Semana", fontsize=16, weight='bold')

# Adiciona os valores em cima de cada barra
for barra in barras:
    altura = barra.get_height()
    plt.text(barra.get_x() + barra.get_width()/2., altura + 0.5,
             f'{int(altura)}', ha='center', va='bottom', fontsize=12)

# Linha da média
plt.axhline(media, color='red', linestyle='--', linewidth=2, label=f'Média: {media:.1f}')
plt.text(len(dias) - 0.5, media + 0.5, f'', color='red', fontsize=12)

# Grade, legenda e layout
plt.grid(True, axis='y', linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()

Fatores Sociodemográficos
Análise: COR DA PELE X CRIME COMETIDO
* Existe alguma correlação entre esses dois fatores?

*Análise feita por: Matheus Kauan*

Nessa análise, serão gerados alguns gráficos para melhor entendimento, mas antees é necessário realizar algumas filtragens e análise dos dados do dataframe

Filtragem do dataframe para seleção de crimes que estão relacionados a violência física/verbal:

In [ ]:
crimes_violencia_fisica_verbal = [
    'LESAO CORPORAL (ART. 129)',
    'LESAO CORPORAL CULPOSA NA DIRECAO DE VEICULO AUTOMOTOR (ART. 303)',
    'LESAO CORPORAL SEGUIDA DE MORTE',
    'AMEACA (ART. 147)',
    'INJURIA (ART. 140)',
    'CALUNIA (ART. 138)',
    'DESACATO (ART. 331)',
    'RESISTENCIA (ART. 329)',
    'VIOLENCIA DOMESTICA',
    'HOMICIDIO DOLOSO',
    'LATROCINIO',
    'MORTE DECORRENTE DE INTERVENCAO POLICIAL'
]

Criando um novo dataframe com a filtragem:

In [ ]:
df_violencia_f_v = df[df['NATUREZA_APURADA'].isin(crimes_violencia_fisica_verbal)].copy()

In [ ]:
#Quantidade de crimes por natureza
print(df_violencia_f_v['NATUREZA_APURADA'].value_counts())

In [ ]:
# Filtrar apenas os registros onde 'COR_PELE' não é nulo
df_violencia_f_v = df_violencia_f_v[df_violencia_f_v['COR_PELE'].notna()]

In [ ]:
print(df_violencia_f_v['COR_PELE'].value_counts())

Verificando a porcentagem de dados que são nulos da coluna 'COR_PELE' em relação ao total de linhas (dados):

In [ ]:
# Número de valores NaN na coluna
num_nan = df['COR_PELE'].isna().sum()

# Número total de linhas
total_linhas = len(df)

# Porcentagem de NaN
porcentagem_nan = (num_nan / total_linhas) * 100

print(f'Porcentagem de NaN em COR_PELE: {porcentagem_nan:.2f}%')

Confirmando a veracidade da porcentagem acima:

In [ ]:
# Filtrar onde COR_PELE é nulo
cor_pele_nulo = df[df['COR_PELE'].isna()]

# Mostrar o dataframe filtrado
print(cor_pele_nulo)

# Mostrar a quantidade de linhas
print(f'Total de linhas com COR_PELE nulo: {len(cor_pele_nulo)}')

In [ ]:
# Filtrar onde COR_PELE não é nulo
cor_pele_preenchido = df[df['COR_PELE'].notna()]

# Mostrar o dataframe filtrado
print(cor_pele_preenchido)

# Mostrar a quantidade de linhas
print(f'Total de linhas com COR_PELE preenchido: {len(cor_pele_preenchido)}')

Aqui encontramos um grande problema: mais de 97% dos dados estão nulos na coluna COR_PELE. Assim, os gráficos serão gerados com base em apenas 3% dos registros que possuem informações sobre a cor da pele da vítima, o que representa um obstáculo para a veracidade e a representatividade das informações que podem ser extraídas dos gráficos a priori.

Criação de um gráfico de barras:

In [ ]:


sns.countplot(data=df_violencia_f_v, x='COR_PELE', hue='NATUREZA_APURADA')
plt.title('Frequência de Crimes por Cor da Pele da Vítima (Baseado em 3% dos dados)')
plt.xlabel('Cor da Pele')
plt.ylabel('Quantidade de Casos')
plt.legend(title='Tipo de Crime')
plt.tight_layout()
plt.show()

In [ ]:
# Listar crimes únicos onde COR_PELE não é nulo
crimes_unicos = df_violencia_f_v['NATUREZA_APURADA'].unique()

print(crimes_unicos)

In [ ]:
#Gerando um HeatMap para visualizar a relação entre COR_PELE e NATUREZA_APURADA

pivot = df_violencia_f_v.pivot_table(index='COR_PELE', columns='NATUREZA_APURADA', aggfunc='size', fill_value=0)
plt.figure(figsize=(12, 6))
sns.heatmap(pivot, annot=True, fmt='d', cmap='Reds')
plt.title('Distribuição de Crimes Verbais/Físicos por Cor da Pele')
plt.xlabel('Tipo de Crime')
plt.ylabel('Cor da Pele')
plt.show()

<div style="text-align: center;">
  <h2>Explorando os dados: datas dos BOs</h2>
  <p>Para entender a natureza dos dias dos acontecimentos e a correlação com datas específicas, cruzaremos os dados de datas atípicas e datas dos crimes. Para realizar esta anlise mais apurada, dividimos os dados em "Natal", "Carnaval", "Demais Feriados", "Jogos de futebol" e "Dias comuns".</p>
</div>

In [ ]:
#Definir as categorias baseadas na sua lista eventos_sp_2022
carnaval = ['2022-02-28', '2022-03-01', '2022-03-02']  # Carnaval + Quarta de Cinzas
natal = ['2022-12-24', '2022-12-25', '2022-12-31']     # Natal + Vésperas

outros_feriados = [
    '2022-01-01', '2022-04-15', '2022-04-21', '2022-05-01',
    '2022-06-16', '2022-07-09', '2022-09-07', '2022-10-12',
    '2022-10-28', '2022-11-02', '2022-11-15'
]

jogos_futebol = [
    '2022-01-25', '2022-03-10', '2022-03-30', '2022-04-03',
    '2022-05-02', '2022-05-22', '2022-06-20', '2022-08-21',
    '2022-09-11', '2022-10-16', '2022-11-24', '2022-11-28',
    '2022-12-02', '2022-12-05', '2022-12-09',  # já existentes

    # Novas datas que não estavam no array original
    '2022-01-30', '2022-02-02', '2022-02-06', '2022-02-17', '2022-02-19',
    '2022-02-20', '2022-02-21', '2022-02-25', '2022-03-17', '2022-03-19',
    '2022-03-20', '2022-04-17', '2022-05-06', '2022-06-19', '2022-07-30',
    '2022-08-06', '2022-09-10', '2022-10-23', '2022-11-12', '2022-12-03'
]


# Função de categorização (ajustada para lidar com strings e datetime)
def categorizar_evento(data):
    if pd.isna(data):  # Tratar valores nulos
        return 'DIA NORMAL'
    
    data_str = data.strftime('%Y-%m-%d') if hasattr(data, 'strftime') else str(data)[:10]  # Adapta para datetime ou string
    if data_str in carnaval:
        return 'CARNAVAL'
    elif data_str in natal:
        return 'NATAL'
    elif data_str in outros_feriados:
        return 'OUTROS FERIADOS'
    elif data_str in jogos_futebol:
        return 'JOGOS FUTEBOL'
    else:
        return 'DIA NORMAL'

# Aplicar a categorização
df['DATA_OCORRENCIA'] = pd.to_datetime(df['DATA_OCORRENCIA'], errors='coerce')  # Garante datetime e trata erros
df['CATEGORIA_EVENTO'] = df['DATA_OCORRENCIA'].apply(categorizar_evento)

# Verificar contagem
print(df['CATEGORIA_EVENTO'].value_counts())

<div style = "text-align: center;">
<p>Agora, vamos comparar a média diária de crimes para cada categoria:</p>
</div>

In [ ]:
# Calcular dias únicos por categoria
dias_por_categoria = {
    'CARNAVAL': len(carnaval),
    'NATAL': len(natal),
    'OUTROS FERIADOS': len(outros_feriados),
    'JOGOS FUTEBOL': len(jogos_futebol),
    'DIA NORMAL': len(df['DATA_OCORRENCIA'].dt.date.unique()) - (len(carnaval) + len(natal) + len(outros_feriados) + len(jogos_futebol))
}

# Calcular média diária
crimes_por_categoria = df.groupby('CATEGORIA_EVENTO').size().reset_index(name='TOTAL_CRIMES')
crimes_por_categoria['MEDIA_DIARIA'] = crimes_por_categoria['TOTAL_CRIMES'] / crimes_por_categoria['CATEGORIA_EVENTO'].map(dias_por_categoria)

# Filtrar apenas eventos (excluindo dias normais)
crimes_analise = crimes_por_categoria[crimes_por_categoria['CATEGORIA_EVENTO'] != 'DIA NORMAL']

# Configurações de estilo (opcional)
sns.set_style("whitegrid")
plt.figure(figsize=(12, 6))

# Cores temáticas para cada categoria
colors = {
    'CARNAVAL': '#FF69B4',  # Rosa
    'NATAL': '#32CD32',     # Verde
    'OUTROS FERIADOS': '#FFA500',  # Laranja
    'JOGOS FUTEBOL': '#1E90FF'     # Azul
}

# Criar o gráfico de barras
bars = plt.bar(
    crimes_analise['CATEGORIA_EVENTO'], 
    crimes_analise['MEDIA_DIARIA'],
    color=[colors[cat] for cat in crimes_analise['CATEGORIA_EVENTO']]
)

# Linha de referência (média diária normal)
media_normal = crimes_por_categoria.loc[crimes_por_categoria['CATEGORIA_EVENTO'] == 'DIA NORMAL', 'MEDIA_DIARIA'].values[0]
plt.axhline(
    media_normal, 
    color='red', 
    linestyle='--', 
    linewidth=2,
    label=f'Média dias normais: {media_normal:.1f}'
)

# Detalhes do gráfico
plt.title('Média Diária de Crimes por Categoria de Evento\nSão Paulo - 2022', fontsize=14, pad=20)
plt.xlabel('Categoria do Evento', fontsize=12)
plt.ylabel('Média de Crimes por Dia', fontsize=12)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.legend(fontsize=11)

# Adicionar valores nas barras
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2, 
        height + 5,  # Offset para posicionar acima da barra
        f'{height:.1f}', 
        ha='center', 
        va='bottom',
        fontsize=10
    )

plt.tight_layout()
plt.show()

<div style = "text-align: center;">
<p>Para uma visão mais detalhada, podemos plotar a quantidade de crimes ao longo do ano, destacando os dias atípicos:</p>
</div>

In [ ]:

# Agrupar crimes por data
crimes_por_data = df.groupby('DATA_OCORRENCIA').size().reset_index(name='TOTAL_CRIMES')

# Identificar dias atípicos no DataFrame
crimes_por_data['DIA_ATIPICO'] = crimes_por_data['DATA_OCORRENCIA'].isin(eventos_dates)

# Configurar o gráfico
plt.figure(figsize=(14, 6))

# Gráfico de linha principal
plt.plot(crimes_por_data['DATA_OCORRENCIA'], 
         crimes_por_data['TOTAL_CRIMES'], 
         label='Crimes por dia', 
         color='gray', 
         alpha=0.6,
         linewidth=1)

# Destacar dias atípicos
dias_atipicos = crimes_por_data[crimes_por_data['DIA_ATIPICO'] == True]
plt.scatter(dias_atipicos['DATA_OCORRENCIA'], 
            dias_atipicos['TOTAL_CRIMES'], 
            color='red', 
            label='Dias atípicos', 
            zorder=5,
            s=50)  # tamanho dos pontos

# Configurar limites do eixo x para 2022
plt.xlim([pd.to_datetime('2022-01-01'), pd.to_datetime('2022-12-31')])

# Formatar eixo x para mostrar meses
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b'))  # Nomes curtos dos meses

# Adicionar detalhes
plt.xlabel('Data (2022)', fontsize=12)
plt.ylabel('Quantidade de Crimes', fontsize=12)
plt.title('Flutuação Diária de Crimes em São Paulo (2022)\nDias Atípicos Destacados', fontsize=14, pad=20)
plt.legend(fontsize=11)
plt.grid(linestyle='--', alpha=0.4)

# Melhorar a legibilidade das datas
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

<div style="text-align: center;">
  <h2>BOs Iniciados X BOs Emitidos</h2>
  <p>Para entender se todos os boletins de ocorrência de fato são emitidos, ou pelo menos uma boa parte, cruzaremos os dados de BO_EMITIDOS e BO_INICIADOS. Tambêm observaremos para quais naturezas de crimes os boletins são mais efeivado e para quais são menos efetivados.</p>
</div>

In [ ]:
#Verificando os tipos de crimes:

if 'NATUREZA_APURADA' in df.columns:
    valores_unicos = df['NATUREZA_APURADA'].dropna().unique()
    print(f"Total de tipos únicos: {len(valores_unicos)}\n")
    print("Exemplos de tipos de natureza apurada:")
    for valor in sorted(valores_unicos)[:50]:  # Mostra os 50 primeiros para não ficar gigante
        print(f"- {valor}")
else:
    print("A coluna 'NATUREZA_APURADA' não foi encontrada no dataset.")

In [ ]:
# 1. Identificar os 6 tipos mais comuns
top_6_crimes = df['NATUREZA_APURADA'].value_counts().head(6).index.tolist()

# 2. Função de categorização
def categorizar_crime(natureza):
    natureza = str(natureza).upper()
    
    # Verificar os 6 tipos mais comuns
    for crime in top_6_crimes:
        if str(crime).upper() in natureza:
            return crime.split('-')[-1].split('(')[0].strip()
    
    # Categorias específicas
    if 'FURTO' in natureza:
        return 'FURTO'
    elif 'ROUBO' in natureza:
        return 'ROUBO'
    elif 'HOMICÍDIO' in natureza or 'HOMICIDIO' in natureza:
        return 'HOMICÍDIO'
    elif 'LESÃO' in natureza or 'LESAO' in natureza:
        return 'LESÃO CORPORAL'
    elif 'ESTELIONATO' in natureza:
        return 'ESTELIONATO'
    elif 'TRÁFICO' in natureza or 'TRAFICO' in natureza:
        return 'TRÁFICO DE DROGAS'
    else:
        return 'OUTROS'

# 3. Aplicar categorização e calcular status
df['CATEGORIA_CRIME'] = df['NATUREZA_APURADA'].apply(categorizar_crime)
df['STATUS_EMISSAO'] = df['BO_EMITIDO'].apply(lambda x: 'Emitido' if pd.notnull(x) else 'Não Emitido')

# 4. Calcular estatísticas por categoria
emissoes_por_categoria = df.groupby('CATEGORIA_CRIME').agg(
    Total_Iniciados=('STATUS_EMISSAO', 'count'),
    Total_Emitidos=('STATUS_EMISSAO', lambda x: (x == 'Emitido').sum())
).reset_index()

emissoes_por_categoria['Perc_Emitidos'] = (emissoes_por_categoria['Total_Emitidos'] / 
                                          emissoes_por_categoria['Total_Iniciados']) * 100

# Ordenar por porcentagem de emissão
emissoes_por_categoria = emissoes_por_categoria.sort_values('Perc_Emitidos', ascending=False)

In [ ]:
# Criar gráfico de barras
plt.figure(figsize=(14, 7))
sns.set_style("whitegrid")

# Paleta de cores - usando as 6 cores principais + cinza para "OUTROS"
cores = sns.color_palette("viridis", n_colors=6) + [(0.7, 0.7, 0.7)]
cores = cores[:len(emissoes_por_categoria)]  # Ajustar para o número real de categorias

ax = sns.barplot(
    data=emissoes_por_categoria,
    x='CATEGORIA_CRIME',
    y='Perc_Emitidos',
    palette=cores,
    order=emissoes_por_categoria['CATEGORIA_CRIME']  # Manter a ordenação
)

# Adicionar rótulos de porcentagem
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2.,
            height + 0.5,
            f'{height:.1f}%',
            ha='center',
            va='bottom',
            fontsize=10)

# Linha de média geral
media_geral = (df['STATUS_EMISSAO'] == 'Emitido').mean() * 100
ax.axhline(media_geral, color='red', linestyle='--', linewidth=1.5)
ax.text(len(emissoes_por_categoria)-0.8, media_geral+2, 
        f'Média Geral: {media_geral:.1f}%', 
        color='red',
        fontsize=11,
        bbox=dict(facecolor='white', alpha=0.8))

# Adicionar volume de ocorrências
for i, row in emissoes_por_categoria.iterrows():
    ax.text(i, -3, 
            f"N={row['Total_Iniciados']:,}", 
            ha='center', 
            va='top', 
            rotation=45, 
            fontsize=9,
            color='dimgrey')

# Configurações do gráfico
plt.title('Taxa de Emissão de BOs por Categoria de Crime\n(Top 6 Tipos Mais Comuns + Outros)', 
          fontsize=15, pad=20)
plt.xlabel('Categoria de Crime', fontsize=12)
plt.ylabel('BOs Emitidos (%)', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=11)
plt.yticks(fontsize=11)
plt.ylim(0, 105)  # Espaço extra para os rótulos

# Ajustar layout
plt.tight_layout()
plt.show()

<div style="text-align: center;">
  <h2>Explorando os dados: Tipologia viária</h2>
  <p>Em quais vias acontecem maior número de crimes? Visando verificar se os dados nos respondem esta pergunta, observaremos aqui a correlação entre o número de crimes e os locais onde aconteceram. Para isso, vamos dividir os dados de LOGRADOURO em: Rua, Avenida, nulo, Rodovia e outros que consite em: Estrada, Praça, Alameda e Travessa; e após isso srá possivel verificar onde mais acontecem crimes</p>
</div>

In [ ]:
# Função para classificar o tipo de logradouro
def classificar_logradouro(log):
    if pd.isna(log):
        return 'NULO'
    log = log.strip().lower()
    if log.startswith('rua'):
        return 'Rua'
    elif log.startswith('avenida') or log.startswith('av.'):
        return 'Avenida'
    elif any(log.startswith(tipo) for tipo in ['praça', 'praca', 'travessa', 'trv.', 'estrada', 'alameda']):
        return 'OUTROS'
    elif log.startswith('rodovia'):
        return 'Rodovia'
    else:
        return 'OUTROS'

# Aplica classificação
df['TIPO_LOGRADOURO'] = df['LOGRADOURO'].apply(classificar_logradouro)

# Contagem por tipo
contagem_logradouros = df['TIPO_LOGRADOURO'].value_counts().reset_index()
contagem_logradouros.columns = ['Tipo de Logradouro', 'Quantidade']

In [ ]:

# Criar gráfico de pizza simples
plt.figure(figsize=(8, 8))
plt.pie(contagem_logradouros['Quantidade'],
        labels=contagem_logradouros['Tipo de Logradouro'],
        autopct='%1.1f%%',
        startangle=90,
        colors=plt.cm.Pastel1.colors,
        wedgeprops={'linewidth': 1, 'edgecolor': 'white'})

# Adicionar título
plt.title('Distribuição de Crimes por Tipo de Logradouro', pad=20)

# Mostrar o gráfico
plt.tight_layout()
plt.show()

<div style="text-align: center;">
  <h2>Crimes em datas de jogos: Inferência estátistica</h2>
  Análise por Arthur Teodoro
  <p>Como já foi observado, as medias de crimes registradas se demonstraram bem mais altas em dias "fora do comum", como feriados e dias de jogos. Visando provar que há relação entre quantidade de crimes e eventos atipícos, faremos aqui demonstrações gráficas e maremáticas que provem o ponto. 
  
  Observe o gráfico a seguir:</p>
</div>

In [ ]:
# Filtrar os dados para dias normais e dias de jogos
df_normal = df[df['CATEGORIA_EVENTO'] == 'DIA NORMAL']
df_futebol = df[df['CATEGORIA_EVENTO'] == 'JOGOS FUTEBOL']

# Contar número de dias únicos por dia da semana (necessário para calcular média)
dias_unicos_normal = df_normal[['DATA_OCORRENCIA', 'DIA_SEMANA']].drop_duplicates()
dias_unicos_futebol = df_futebol[['DATA_OCORRENCIA', 'DIA_SEMANA']].drop_duplicates()

# Contar crimes por dia da semana
crimes_normal = df_normal.groupby('DIA_SEMANA').size()
crimes_futebol = df_futebol.groupby('DIA_SEMANA').size()

# Contar dias únicos por dia da semana
contagem_dias_normal = dias_unicos_normal['DIA_SEMANA'].value_counts()
contagem_dias_futebol = dias_unicos_futebol['DIA_SEMANA'].value_counts()

# Calcular média de crimes por dia da semana
media_normal = (crimes_normal / contagem_dias_normal).fillna(0)
media_futebol = (crimes_futebol / contagem_dias_futebol).fillna(0)

# Juntar os dois em um DataFrame
comparativo = pd.DataFrame({
    'Dias Normais': media_normal,
    'Dias com Jogos': media_futebol
}).fillna(0)

# Ordenar os dias da semana corretamente
ordem_dias = [
    'SEGUNDA-FEIRA', 'TERÇA-FEIRA', 'QUARTA-FEIRA',
    'QUINTA-FEIRA', 'SEXTA-FEIRA', 'SÁBADO', 'DOMINGO'
]
comparativo = comparativo.reindex(ordem_dias)

# Plotar gráfico de barras lado a lado
plt.figure(figsize=(12, 6))
comparativo.plot(kind='bar', width=0.8, color=['#808080', '#1E90FF'], edgecolor='black')

plt.title('Média de Crimes por Dia da Semana\nComparação: Dias Normais vs Dias com Jogos de Futebol', fontsize=14, pad=20)
plt.xlabel('Dia da Semana', fontsize=12)
plt.ylabel('Média de Crimes por Dia', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=11)
plt.yticks(fontsize=11)
plt.legend(fontsize=11)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


Como é possível ver no gráfico, as médias de crimes em dias com jogos marcados são, em alguns casos, maiores do que as médias em dias comuns, para todos os dias da semana. Mas, apesar de a interferência do fator 'partida de futebol' já estar bastante evidente, analisaremos também estatisticamente usando o método de hipóteses.

### Teste de Hipótese: Impacto de Jogos de Futebol na Média de Crimes

Neste teste, buscamos avaliar se há uma diferença estatisticamente significativa na **média de crimes registrados em dias com jogos de futebol** em comparação com **dias normais**, em São Paulo no ano de 2022.

### Hipóteses

- H₀ (hipótese nula): A média de crimes em dias com jogos é igual à média de dias normais.
- H₁ (hipótese alternativa): A média de crimes em dias com jogos é **maior** do que a média de dias normais.


Utilizamos um **teste de hipótese para diferença de médias** com base na distribuição normal (z-teste), assumindo:
- Os dados seguem uma distribuição aproximadamente normal (via Teorema Central do Limite).
- As amostras são independentes.

O valor de **z-score** é calculado pela fórmula:

$$
z = \frac{\bar{x}_1 - \bar{x}_2}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}
$$

Onde:
- \( $\bar{x}_1$, $s_1$, $n_1$ \) são média, desvio padrão e tamanho da amostra dos **dias com jogos**
- \( $\bar{x}_2$, $s_2$, $n_2$ \) são os mesmos para **dias normais**

Em seguida, comparamos o **valor-p** ao nível de significância usual de 5% \( $\alpha$ = 0.05 \). Se o valor-p for menor que 0.05, rejeitamos a hipótese nula.


In [ ]:
from scipy.stats import norm

# Agrupar total de crimes por dia
crimes_por_dia = df.groupby(['DATA_OCORRENCIA', 'CATEGORIA_EVENTO']).size().reset_index(name='TOTAL_CRIMES')

# Separar amostras
crimes_normais = crimes_por_dia[crimes_por_dia['CATEGORIA_EVENTO'] == 'DIA NORMAL']['TOTAL_CRIMES']
crimes_jogos = crimes_por_dia[crimes_por_dia['CATEGORIA_EVENTO'] == 'JOGOS FUTEBOL']['TOTAL_CRIMES']

# Parâmetros das amostras
x1 = crimes_jogos.mean()
x2 = crimes_normais.mean()
s1 = crimes_jogos.std(ddof=1)
s2 = crimes_normais.std(ddof=1)
n1 = len(crimes_jogos)
n2 = len(crimes_normais)

# Estatística z
z = (x1 - x2) / np.sqrt((s1**2)/n1 + (s2**2)/n2)

# Valor-p (teste unilateral à direita)
p_valor = 1 - norm.cdf(z)

# Mostrar resultados
print(f"Média (jogos): {x1:.2f}")
print(f"Média (normais): {x2:.2f}")
print(f"Z-score: {z:.2f}")
print(f"Valor-p: {p_valor:.4f}")

# Conclusão
alpha = 0.05
if p_valor < alpha:
    print("➡️ Rejeitamos H₀: Há evidência de que a média de crimes em dias com jogos é maior.")
else:
    print("➡️ Não rejeitamos H₀: Não há evidência suficiente de que a média seja maior.")


Conforme vimos acima, o teste de hipótese está em concordância com os gráficos apresentados anteriormente. Portanto, com o resultado obtido, **é possível afirmar que a ocorrência de uma partida de futebol em determinada data pode repercutir em um notável aumento no número de crimes**.

<div style="text-align: center;">
  <h2>O crime folga aos fins de semana: inferência estatística</h2>
  Análise por Arthur Teodoro
  <p>Ficou provado que os números de crimes aumentam dependendo se há ou não partidas de futebol na data, mas há diferença com relação ao dia da semana em que esses jogos ocorrem? Jogos no meio da semana tem ainda mais crimes que os jogos de sábado e domingo? A seguir análisaremos, novamente, de forma gráfica e estatística, se há uma correlação
  Observe o gráfico a seguir:</p>
</div>

In [ ]:
# Garantir datetime
df['DATA_OCORRENCIA'] = pd.to_datetime(df['DATA_OCORRENCIA'], errors='coerce')

# Criar coluna com dia da semana em português
dias_semana = {
    'Monday': 'SEGUNDA-FEIRA', 'Tuesday': 'TERÇA-FEIRA', 'Wednesday': 'QUARTA-FEIRA',
    'Thursday': 'QUINTA-FEIRA', 'Friday': 'SEXTA-FEIRA', 'Saturday': 'SÁBADO', 'Sunday': 'DOMINGO'
}
df['DIA_SEMANA'] = df['DATA_OCORRENCIA'].dt.day_name().map(dias_semana)

# Adicionar classificação "FIM DE SEMANA" ou "DIA ÚTIL"
df['TIPO_DIA'] = df['DIA_SEMANA'].apply(lambda x: 'FIM DE SEMANA' if x in ['SÁBADO', 'DOMINGO'] else 'DIA ÚTIL')

# Filtrar apenas os dias com jogos
df_jogos = df[df['CATEGORIA_EVENTO'] == 'JOGOS FUTEBOL']

# Agrupar por data para obter número de crimes por dia
crimes_por_dia_jogo = df_jogos.groupby(['DATA_OCORRENCIA', 'TIPO_DIA']).size().reset_index(name='QTD_CRIMES')

# Calcular média por tipo de dia
media_crimes = crimes_por_dia_jogo.groupby('TIPO_DIA')['QTD_CRIMES'].mean().reset_index()

# Plotar
plt.figure(figsize=(8, 6))
sns.barplot(data=media_crimes, x='TIPO_DIA', y='QTD_CRIMES', palette=['#1E90FF', '#FF8C00'])

# Adicionar rótulos
for i, row in media_crimes.iterrows():
    plt.text(i, row['QTD_CRIMES'] + 1, f'{row["QTD_CRIMES"]:.1f}', ha='center', fontsize=11)

plt.title('Média de Crimes em Dias com Jogos de Futebol\nComparando Fins de Semana e Dias Úteis', fontsize=14, pad=20)
plt.xlabel('Tipo de Dia', fontsize=12)
plt.ylabel('Média de Crimes por Dia com Jogo', fontsize=12)
plt.tight_layout()
plt.show()


Considerando apenas sábado e domingo como fim de semana, e os demais dias como meio de semana, não é possível observar claramente uma grande diferença entre as médias representadas nas barras. Portanto, realizaremos novamente um teste de hipótese para verificar se há uma diferença significativa entre o fim de semana e o meio de semana.

### Hipóteses:

- **H₀ (nula):** $\mu_{\text{útil}} = \mu_{\text{fim\_semana}}$ (mesma média de crimes)
- **H₁ (alternativa):** $\mu_{\text{útil}} \neq \mu_{\text{fim\_semana}}$ (diferença significativa)

Em seguida, comparamos o **valor-p** ao nível de significância usual de 5% \( $\alpha$ = 0.05 \). Se o valor-p for menor que 0.05, rejeitamos a hipótese nula.

In [ ]:
# Reutilizando os dados de dias com jogos
df_jogos = df[df['CATEGORIA_EVENTO'] == 'JOGOS FUTEBOL']

# Criar coluna com tipo de dia
df_jogos['TIPO_DIA'] = df_jogos['DIA_SEMANA'].apply(lambda x: 'FIM DE SEMANA' if x in ['SÁBADO', 'DOMINGO'] else 'DIA ÚTIL')

# Agrupar por data e tipo de dia
crimes_por_dia = df_jogos.groupby(['DATA_OCORRENCIA', 'TIPO_DIA']).size().reset_index(name='QTD_CRIMES')

# Separar os dois grupos
grupo_util = crimes_por_dia[crimes_por_dia['TIPO_DIA'] == 'DIA ÚTIL']['QTD_CRIMES']
grupo_fds = crimes_por_dia[crimes_por_dia['TIPO_DIA'] == 'FIM DE SEMANA']['QTD_CRIMES']

# Estatísticas básicas
media_util = grupo_util.mean()
media_fds = grupo_fds.mean()
std_util = grupo_util.std(ddof=1)
std_fds = grupo_fds.std(ddof=1)
n_util = len(grupo_util)
n_fds = len(grupo_fds)

# Estatística z
z = (media_util - media_fds) / np.sqrt((std_util**2 / n_util) + (std_fds**2 / n_fds))

# Valor-p (teste bilateral)
p_valor = 2 * (1 - norm.cdf(abs(z)))

# Exibir resultados
print(f"Média (dia útil): {media_util:.2f}")
print(f"Média (fim de semana): {media_fds:.2f}")
print(f"Z-score: {z:.2f}")
print(f"Valor-p: {p_valor:.4f}")


Desta vez, o valor-p é maior que 0,05, ou seja, não rejeitamos a hipótese nula (H₀).
Após a análise do teste de hipótese, concluímos que não há uma diferença significativa no número de crimes entre os dias de fim de semana e de meio de semana, e **não é possível afirmar que um período tende a apresentar mais ocorrências do que o outro**.

<div style="text-align: center;">
  <h2>Associações</h2>
  <p>Abaixo exploraremos algumas associações para que possamos interpretar melhor os dados, e encontrar alguns padrões que posso ajudar a entendê-los </p>
</div>

Pensando em regras de associação, irá ser feita uma análise de que turno do dia um crime é realizado em um determinado, com base na natureza apurada, turno e local. (Matheus Kauan)

In [ ]:
# Calcula a porcentagem de valores não nulos
porcentagem_nao_nulos = df['NATUREZA_APURADA'].notna().mean() * 100

# Exibe a porcentagem
print(f"{porcentagem_nao_nulos:.2f}% dos dados de NATUREZA_APURADA não são nulos.")

In [ ]:
# Calcula a porcentagem de valores não nulos
porcentagem_nao_nulos = df['TIPO_LOCAL'].notna().mean() * 100

# Exibe a porcentagem
print(f"{porcentagem_nao_nulos:.2f}% dos dados de TIPO_LOCAL não são nulos.")

In [ ]:
# Calcula a porcentagem de valores não nulos
porcentagem_nao_nulos = df['PERIODO_OCORRENCIA'].notna().mean() * 100

# Exibe a porcentagem
print(f"{porcentagem_nao_nulos:.2f}% dos dados de PERIODO_OCORRENCIA não são nulos.")

Após as verificações, podemos realizar as análises visto que maior parte dos dados possuem todos os atributos necessários

In [ ]:
# Seleciona e faz uma cópia explícita das colunas
df_assoc = df[['NATUREZA_APURADA', 'TIPO_LOCAL', 'PERIODO_OCORRENCIA']].copy()

# Agora é seguro remover os nulos sem warnings
df_assoc.dropna(inplace=True)


In [ ]:
# Instala automaticamente o mlxtend, se não estiver instalado
try:
    from mlxtend.preprocessing import TransactionEncoder
    from mlxtend.frequent_patterns import apriori, association_rules
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "mlxtend"])
    from mlxtend.preprocessing import TransactionEncoder
    from mlxtend.frequent_patterns import apriori, association_rules


In [ ]:
df_assoc['itens'] = df_assoc.apply(lambda row: [
    f"NATUREZA={row['NATUREZA_APURADA']}",
    f"LOCAL={row['TIPO_LOCAL']}",
    f"PERIODO={row['PERIODO_OCORRENCIA']}"
], axis=1)

In [ ]:

te = TransactionEncoder()
te_ary = te.fit(df_assoc['itens']).transform(df_assoc['itens'])
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
# Gera itemsets frequentes com suporte mínimo de 1%
freq_items = apriori(df_encoded, min_support=0.01, use_colnames=True)

# Gera regras com confiança mínima de 60%
regras = association_rules(freq_items, metric="confidence")

# Exibe as 10 primeiras
regras[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

In [ ]:
regras.shape[0]

In [ ]:
df_assoc

In [ ]:
df.head(5)


Arquivo CSV atualizado:

In [ ]:
df2 = pd.read_csv('dados/SpSafe_2022(c_cidades).csv', delimiter=';')
pd.set_option('display.max_columns', None)

<p>Agora tentaramos ver se é valido o investimento em efetivo policial nas sextas e sábados a noite a fim de uma melhor prevenção ao crime (Gabriel) </p>

In [ ]:
# Calcular a porcentagem de valores não nulos
porcentagem_nao_nulos = df['DIA_SEMANA'].notna().mean() * 100

print(f"{porcentagem_nao_nulos:.2f}% dos dados de DIA_SEMANA não são nulos.")

<p> Como 99,91% dos dados DIA_SEMANA não são nulos, irei descartar os que são, e construir a tabela de transação com os disponiveis </p> 

In [ ]:
# Converte para maiúsculas (caso os dados estejam com letras minúsculas ou misturadas)
df['DIA_SEMANA'] = df['DIA_SEMANA'].str.upper()
df['PERIODO_OCORRENCIA'] = df['PERIODO_OCORRENCIA'].str.upper()

# Filtra os dados com base nos critérios
df_filtrado = df.dropna(subset=['DIA_SEMANA', 'PERIODO_OCORRENCIA', 'NATUREZA_APURADA'])



<p> Usando o algoritmo apriori para validar (ou não) as associaçoes </p>

In [ ]:
# Fazendo um filtro com as colunas envolvidas 
transacoes = df_filtrado.apply(lambda row: [
    f"DIA_SEMANA={row['DIA_SEMANA']}",
    f"PERIODO_OCORRENCIA={row['PERIODO_OCORRENCIA']}",
    f"NATUREZA_APURADA={row['NATUREZA_APURADA']}"
], axis=1).tolist()


In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# 1. Montar as transações apenas com DIA_SEMANA e PERIODO_OCORRENCIA
transacoes_temporais = df_filtrado.apply(lambda row: [
    f"DIA_SEMANA={row['DIA_SEMANA']}",
    f"PERIODO_OCORRENCIA={row['PERIODO_OCORRENCIA']}"
], axis=1).tolist()

# 2. Codificar as transações
te_temp = TransactionEncoder()
te_temp_ary = te_temp.fit(transacoes_temporais).transform(transacoes_temporais)
df_encoded_temp = pd.DataFrame(te_temp_ary, columns=te_temp.columns_)

# 3. Rodar Apriori
frequent_itemsets_temp = apriori(df_encoded_temp, min_support=0.01, use_colnames=True)

# 4. Gerar regras de associação
regras_temp = association_rules(frequent_itemsets_temp, metric="confidence", min_threshold=0.1)

# 5. Deixar os resultados mais legíveis
regras_temp['antecedents'] = regras_temp['antecedents'].apply(lambda x: ', '.join(list(x)))
regras_temp['consequents'] = regras_temp['consequents'].apply(lambda x: ', '.join(list(x)))

# 6. Selecionar e ordenar
regras_temporais_formatadas = regras_temp[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
regras_ordenadas = regras_temporais_formatadas.sort_values(by='lift', ascending=False)

# 7. Mostrar as top regras
print(regras_ordenadas.head(10))


<p> Com a execução do algoritmo temos: </p>

In [ ]:
# Filtrar regras onde antecedents é 'DIA_SEMANA=SEXTA-FEIRA' e consequents é 'PERIODO_OCORRENCIA=A NOITE'
filtro = (
    ((regras_ordenadas['antecedents'] == 'DIA_SEMANA=SEXTA-FEIRA') |
     (regras_ordenadas['antecedents'] == 'DIA_SEMANA=SÁBADO')) &
    (regras_ordenadas['consequents'] == 'PERIODO_OCORRENCIA=A NOITE')
)
regras_ordenadas[filtro]

In [ ]:
# Ordena as regras pelo suporte de forma decrescente
regras_ordenadas = regras_ordenadas.sort_values(by='support', ascending=False)
regras_ordenadas

In [ ]:
media_support = regras_ordenadas['support'].mean()
print(f"Média do suporte: {media_support:.4f}")
mediana_support = regras_ordenadas['support'].median()
print(f"Mediana do suporte: {mediana_support:.4f}")

<p> Analisando os dados, vi que aproximadamente associando sexta-feira e sábado com o total, 5% de todas as ocorrência ocorreram em cada  um desses dias. e Desse sub-conjunto, aproximadamente 30% ocorreram a noite, com o lift beirando a estabilidade. Vendo o conjunto de dados por completo, percebi que que a distribuição dos crimes pelo dia da semana está bem normalizada, tendo a média e a mediana aproximadamente com 3%. Sendo assim, não valeria a pena o investimento em efetivo policial em determinado dia da semana e periodo como forma prevenção a crime.</p>  

<div style="text-align: center;">
  <h3>Tipos de crimes para com locais de elaboração</h3>
  <p>Estudo para descobrir se há associação entre o tipo de vrime que a pessoa passou para com o tipo de delegacia que foi elaborado o B.O</p>
</div>

Verificando se há algum ruído nos campos que serão utilizados para realizar o estudo de associatividade

In [ ]:
porcentagem_crimes = df['NATUREZA_APURADA'].notna().mean() * 100
print(f"{porcentagem_crimes:.2f}% das linhas possuem 'NATUREZA_APURADA' preenchida.")

porcentagem_elab = df['LOCAL_ELABORACAO'].notna().mean() * 100
print(f"{porcentagem_elab:.2f}% das linhas possuem 'LOCAL_ELABORACAO' preenchida.")


Criação de grupos de tipos de delegacias e tipos de crimes semelhantes(podendo pertencer a algum grupo). Foi feito isso na intenção de diminuir a quantidade de itens e itensets possíveis.

In [ ]:
grupo_delegacia = {
    "Delegacia Distrital (DP)": r"\d+º D\.P\.",
    "Delegacia de Plantão": r"PLANTAO|PLANTÃO",
    "Delegacia da Mulher (DDM)": r"DDM",
    "Delegacia de Homicídios (DHPP/DIG)": r"DHPP|DIG|HOMICIDIOS",
    "Delegacia de Narcóticos (DISE/DENARC)": r"DISE|DENARC|ENTORPECENTES",
    "Delegacia de Roubos (DRADE)": r"DRADE|ROUBO|FURTO",
    "Delegacia de Crimes Financeiros (DCCIBER)": r"DCCIBER|FRAUDE|FINANCEIRO",
    "Delegacia do Idoso": r"PROTECAO IDOSO|IDOSO",
    "Delegacia da Criança/Adolescente": r"INF\.JUV|JUVENIL|CRIANÇA",
    "Delegacia de Capturas (DIPE)": r"DIPE|CAPTURA|FUGA",
    "Delegacia Ambiental (DICCA)": r"DICCA|AMBIENTAL",
    "Delegacia da Fazenda (DIIMA)": r"DIIMA|FAZENDA|SONEGAÇÃO",
    "Delegacia Municipal (DEL.POL.)": r"^DEL\.POL\.",
    "Delegacia Eletrônica": r"ELETRONICA|ONLINE",
    "Circunscrição Policial (CPJ)": r"CPJ",
    "Outras": r".*"  # Padrão genérico (caso não se encaixe em nenhum outro)
}

def classificar_delegacia(delegacia):
    for grupo, regex in grupo_delegacia.items():
        if re.search(regex, delegacia, re.IGNORECASE):
            return grupo
    return "Outras"  # Caso não encontre nenhum padrão

In [ ]:
import re

grupos_crime = {
    # 1. Crimes Violentos Contra a Vida
    "Homicídio Doloso": r"HOMICIDIO DOLOSO|HOMICIDIO SIMPLES|HOMICIDIO QUALIFICADO",
    "Morte Decorrente de Intervenção Policial": r"MORTE DECORRENTE DE INTERVENCAO POLICIAL",
    "Lesão Corporal Seguida de Morte": r"LESAO CORPORAL SEGUIDA DE MORTE",
    "Latrocínio": r"LATROCINIO",
    
    # 2. Roubos (subdivididos)
    "Roubo a Pessoa": r"ROUBO.*TRANSEUNTE|ROUBO.*SAIDINHA",
    "Roubo a Veículos": r"ROUBO.*VEICULO|ROUBO.*CARGA",
    "Roubo a Estabelecimentos": r"ROUBO.*ESTABELECIMENTO|ROUBO.*JOALHERIA",
    "Roubo em Residências": r"ROUBO.*RESIDENCIA|ROUBO.*CONDOMINIO",
    
    # 3. Furtos (subdivididos)
    "Furto a Veículos": r"FURTO.*VEICULO|FURTO.*INTERIOR DE VEICULO",
    "Furto a Estabelecimentos": r"FURTO.*ESTABELECIMENTO|FURTO.*JOALHERIA",
    "Furto em Residências": r"FURTO.*RESIDENCIA|FURTO.*CONDOMINIO",
    "Furto Qualificado": r"FURTO QUALIFICADO",
    
    # 4. Crimes Sexuais
    "Estupro": r"ESTUPRO|ART\. 213",
    "Importunação Sexual": r"IMPORTUNACAO SEXUAL|ASSEDIO SEXUAL",
    "Crimes Sexuais contra Vulneráveis": r"ESTUPRO DE VULNERAVEL|CORRUPCAO DE MENORES",
    
    # 5. Crimes contra o Patrimônio (outros)
    "Extorsão": r"EXTORSAO",
    "Fraudes": r"ESTELIONATO|FRAUDE|MOEDA FALSA",
    "Dano": r"DANO|DANO QUALIFICADO",
    
    # 6. Crimes contra a Pessoa
    "Lesão Corporal": r"LESAO CORPORAL",
    "Ameaça": r"AMEACA|PERSEGUIR",
    "Violência Doméstica": r"VIOLENCIA DOMESTICA|VIOLENCIA PSICOLOGICA",
    
    # 7. Crimes de Trânsito
    "Embriaguez ao Volante": r"EMBRIAGUEZ AO VOLANTE",
    "Acidentes de Trânsito": r"ACIDENTE|ATROPELAMENTO|COLISAO",
    "Direção Perigosa": r"DIRECAO PERIGOSA|VELOCIDADE INCOMPATIVEL",
    
    # 8. Crimes contra a Administração Pública
    "Corrupção": r"CORRUPCAO",
    "Falsificação": r"FALSIDADE|FALSIFICACAO",
    
    # 9. Crimes Digitais
    "Crimes Cibernéticos": r"INFORMATICA|INVASAO DE DISPOSITIVO",
    
    # 10. Outros Crimes
    "Porte Ilegal de Arma": r"PORTE ILEGAL|POSSE ILEGAL DE ARMA",
    "Tráfico de Drogas": r"DROGAS.*AUTORIZACAO|ENTORPECENTES",
    "Outros Crimes": r".*"  # Padrão genérico para casos não classificados
}

def classificar_crime(crime):
    for grupo, regex in grupos_crime.items():
        if re.search(regex, crime, re.IGNORECASE):
            return grupo
    return "Outros Crimes"  # Caso não encontre nenhum padrão

In [ ]:
import re

df['GRUPO_ELAB'] = df['LOCAL_ELABORACAO'].apply(classificar_delegacia)
df['GRUPO_CRIME'] = df['NATUREZA_APURADA'].apply(classificar_crime)
df

In [ ]:
df_elab_group = df[['NUM_BO','GRUPO_CRIME', 'GRUPO_ELAB']].copy()
df_elab_group.dropna(inplace=True)
df_elab_group

In [ ]:
df_elab = df[['NUM_BO','NATUREZA_APURADA','LOCAL_ELABORACAO']].copy()
df_elab.dropna(inplace=True)
df_elab

In [ ]:
df_elab_group['itens'] = df_elab_group.apply(lambda row: [row['GRUPO_CRIME'], row['GRUPO_ELAB']], axis=1)
print(df_elab_group[['itens']].head(5))

In [ ]:

df_elab['itens'] = df_elab.apply(lambda row: [row['NATUREZA_APURADA'], row['LOCAL_ELABORACAO']], axis=1)
print(df_elab[['itens']].head(5))

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

# Inicializar e transformar os dados
te = TransactionEncoder()
te_ary = te.fit(df_elab['itens']).transform(df_elab['itens'])

# Criar DataFrame codificado
df_encoded_elab = pd.DataFrame(te_ary, columns=te.columns_)
df_encoded_elab

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

# Inicializar e transformar os dados
te = TransactionEncoder()
te_ary = te.fit(df_elab_group['itens']).transform(df_elab_group['itens'])

# Criar DataFrame codificado
df_encoded_elab_group = pd.DataFrame(te_ary, columns=te.columns_)
df_encoded_elab_group

In [ ]:
from mlxtend.frequent_patterns import apriori

# Encontrar itens frequentes (pares crime + delegacia)
frequent_itemsets = apriori(
    df_encoded_elab,
    min_support=0.05,  # Ajuste conforme seu dataset (ex: 5% das ocorrências)
    use_colnames=True
)

# Ordenar por suporte
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
frequent_itemsets

In [ ]:
from mlxtend.frequent_patterns import apriori

# Encontrar itens frequentes (pares crime + delegacia)
frequent_itemsets = apriori(
    df_encoded_elab_group,
    min_support=0.05,  # Ajuste conforme seu dataset (ex: 5% das ocorrências)
    use_colnames=True
)

# Ordenar por suporte
frequent_itemsets_group = frequent_itemsets.sort_values(by='support', ascending=False)
frequent_itemsets_group

Com a finalidade de entender se há alguma relação entre crimes e locais de leaboração a confiança foi ajustada para zero, mesmo assim houve apenas poucos casos de associação(repetidos), mesmo juntando as duas abordagens(agrupada e não agrupada). Embora exista o seu grau de confiança é muito baixo, além de seu grau de associação também ser baixo e o lift alto. *Necessário uma busca para entender o que estes dados estão querendo dizer*.

In [ ]:
from mlxtend.frequent_patterns import association_rules

# Gerar regras com base na confiança ou lift
rules = association_rules(
    frequent_itemsets,
    metric='confidence',
    min_threshold=0.00  # Confiança mínima de 0%
)

# Filtrar regras relevantes (ex: lift > 1.5)
rules[rules['lift'] >= 0.01].sort_values(by='lift', ascending=False)

In [ ]:
from mlxtend.frequent_patterns import association_rules

# Gerar regras com base na confiança ou lift
rules = association_rules(
    frequent_itemsets_group,
    metric='confidence',
    min_threshold=0.00  # Confiança mínima de 0%
)

# Filtrar regras relevantes (ex: lift > 1.5)
rules[rules['lift'] >= 0.01].sort_values(by='lift', ascending=False)

In [ ]:
# Futebol (datas de jogos importantes de times paulistas em 2022)
jogos_futebol = [
    '2022-01-27',  # Athletico-PR 1-1 São Paulo
    '2022-02-20',  # Internacional 2-1 Corinthians
    '2022-03-06',  # Flamengo 1-1 Palmeiras
    '2022-04-02',  # São Paulo 2-1 Palmeiras (Paulistão)
    '2022-04-10',  # Corinthians 1-0 Flamengo
    '2022-04-17',  # Atlético-MG 0-0 Santos
    '2022-05-15',  # Fluminense 2-0 Corinthians
    '2022-05-22',  # São Paulo 1-1 Palmeiras
    '2022-06-19',  # Fortaleza 0-3 Palmeiras
    '2022-07-10',  # Botafogo 1-2 São Paulo
    '2022-07-13',  # São Paulo 3-1 Juventude (Copa do Brasil)
    '2022-08-07',  # Goiás 1-0 Santos
    '2022-08-13',  # Corinthians 0-2 Flamengo (Copa do Brasil)
    '2022-08-28',  # Palmeiras 4-2 Corinthians
    '2022-09-04',  # Coritiba 0-1 Palmeiras
    '2022-09-17',  # Santos 0-1 São Paulo
    '2022-10-02',  # Cuiabá 1-1 Corinthians
    '2022-11-06',  # Avaí 1-3 São Paulo
    '2022-11-24',  # Corinthians 2-0 São Paulo
    '2022-11-28',  # Corinthias 1-0 Athetico-PR
    '2022-12-02',  # América-MG 1-2 Palmeiras
    '2022-12-05',  # São Paulo 4-0 Fluminense
    '2022-12-09'   # Santos 2-1 Botafogo
]

Adicionando o campo de 'Houve_JOGO' no dataset, para melhorar a manipulação dos dados

In [ ]:
df['HOUVE_JOGO'] = df['DATA_OCORRENCIA'].dt.strftime('%Y-%m-%d').isin(jogos_futebol)
df

Definindo apenas quais campos serão utilizados para verificar sua assosiatividade

In [ ]:
df_jogos = df[['NATUREZA_APURADA', 'HOUVE_JOGO']].copy()
df_jogos.dropna(inplace=True)
df_jogos

Colocando no formato de transições

In [ ]:
df_jogos['itens'] = df_jogos.apply(lambda row: [row['NATUREZA_APURADA'], row['HOUVE_JOGO']], axis=1)
print(df_jogos[['itens']])

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

# Criar transações no formato: [['ROUBO', 'SIM_JOGO'], ['FURTO', 'NAO_JOGO'], ...]
transacoes = df_jogos.apply(
    lambda row: [row['NATUREZA_APURADA'], 'SIM_JOGO' if row['HOUVE_JOGO'] else 'NAO_JOGO'], 
    axis=1
).tolist()

# Codificar como matriz binária
te = TransactionEncoder()
te_array = te.fit_transform(transacoes)
df_encoded_jogos = pd.DataFrame(te_array, columns=te.columns_)

Aplicando o algoritmo:

In [ ]:
from mlxtend.frequent_patterns import apriori

# Itens frequentes (suporte mínimo de 5%)
frequent_items_jogos = apriori(df_encoded_jogos, min_support=0.05, use_colnames=True)
frequent_items_jogos

In [ ]:
from mlxtend.frequent_patterns import association_rules

# Regras com confiança mínima de 70%
regras = association_rules(frequent_items_jogos, metric="confidence", min_threshold=0.0001)
regras[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

<div style="text-align: center;">
    <h3> Jogos de futebol com tipos de crimes </h3>
    <p> Os jogos de futebol serão usados aqueles que tiverem algum time paulista </p>
</div>

<div style="text-align: center;">
    <h1> Arquivo csv atualizado </h1>
    <p> Executar a célula somente se não tiver o arquivo .csv do SpSafe atualizado.  </p>
</div>

In [ ]:
# Tenta importar gdown, se não conseguir, instala
try:
    import gdown  # Tenta importar novamente após a instalação
except ImportError:
    print("📥 gdown não encontrado. Instalando automaticamente...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])

# Baixar o arquivo do Google Drive

file_id = "1Tf6-immhPLj23A0Lvai52j-WQpl6KjKt"
url = f"https://drive.google.com/uc?id={file_id}"
output = "dados/SpSafe_2022(c_cidades).csv"

gdown.download(url, output, quiet=False)